In [1]:
from pathlib import Path

import nibabel as nib

In [2]:
import nibabel as nib
import numpy as np
import os

# Chemin de votre image originale
input_path = "/home/tibia/Documents/Brain_GPT_Dataset/ID_0e0fbb4d_ID_e7fa3a566c.nii.gz"
output_path = "/home/tibia/Documents/Brain_GPT_Dataset/black_image.nii.gz" # À personnaliser

# 1. Charger l'image pour récupérer les métadonnées (forme et affine)
original_img = nib.load(input_path)
data_shape = original_img.shape
data_affine = original_img.affine
data_dtype = original_img.get_data_dtype() # Récupérer le type de données (ex: int16, float32)
black_data = np.zeros(data_shape, dtype=data_dtype)
black_img = nib.Nifti1Image(black_data, data_affine, header=original_img.header)

# Optionnel: Mettre à jour le header si nécessaire, ou le copier simplement (fait ci-dessus)

# 4. Sauvegarder la nouvelle image noire au format .nii.gz
nib.save(black_img, output_path)

print(f"Image noire sauvegardée avec succès à : {output_path}")
print(f"Forme des données : {data_shape}")

Image noire sauvegardée avec succès à : /home/tibia/Documents/Brain_GPT_Dataset/black_image.nii.gz
Forme des données : (512, 512, 30)


In [2]:
from pathlib import Path

New_DIR= Path("/home/tibia/Documents/Brain_GPT_Dataset")
nii_files = list(New_DIR.glob("*.nii.gz"))

for file in nii_files:
    img = nib.load(str(file))
    data = img.get_fdata()
    shape = data.shape
    num_slices = shape[2] if len(shape) == 3 else "Non applicable"

    print(f"Fichier: {file.name}")
    print(f"  - Shape: {shape}")
    print(f"  - Nombre de coupes (axe Z): {num_slices}")
    print(f"  - Affine:\n{img.affine}")
    print("-" * 40)



Fichier: ID_0e0fbb4d_ID_e7fa3a566c.nii.gz
  - Shape: (512, 512, 30)
  - Nombre de coupes (axe Z): 30
  - Affine:
[[ -0.48828101   0.           0.         125.        ]
 [  0.          -0.45272622  -2.02007361 144.98588562]
 [  0.          -0.18291342   4.99985329  16.0782795 ]
 [  0.           0.           0.           1.        ]]
----------------------------------------
Fichier: ID_00526c11_ID_d6296de728.nii.gz
  - Shape: (512, 512, 29)
  - Nombre de coupes (axe Z): 29
  - Affine:
[[ -0.48828125   0.           0.         125.        ]
 [  0.          -0.43886428  -2.43314394  21.29016113]
 [  0.          -0.21404842   4.98868425 -60.3269577 ]
 [  0.           0.           0.           1.        ]]
----------------------------------------
Fichier: ID_1423bd41_ID_72db71a879.nii.gz
  - Shape: (512, 512, 32)
  - Nombre de coupes (axe Z): 32
  - Affine:
[[ -0.494863     0.           0.         126.43737793]
 [  0.          -0.494863     0.         126.43737793]
 [  0.           0.        

In [ ]:
import nibabel as nib
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

INPUT_DIR = Path("/home/tibia/Documents/Brain_GPT_Dataset/black")
SLICE_OUTPUT_DIR = INPUT_DIR / "slices_png"
SLICE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

nii_files = list(INPUT_DIR.glob("*.nii.gz"))

for file in nii_files:
    img = nib.load(str(file))
    data = img.get_fdata()  # shape (H, W, D) où D > 24
    base_name = file.stem  # sans extension

    # Le nombre de tranches total dans le volume
    total_slices = data.shape[2]
    
    # Vérification: s'il y a moins de 24 tranches, on ne peut pas en sélectionner 24.
    if total_slices < 24:
        print(f"  {file.name} ignoré (seulement {total_slices} tranches)")
        continue


    selected_indices = np.linspace(0, total_slices - 1, 24).astype(int)

    # Sauvegarde les slices sélectionnées
    for i, slice_index in enumerate(selected_indices):
        slice_2d = data[:, :, slice_index]
        
        # Normalisation simple [0, 255]
        slice_norm = 255 * (slice_2d - np.min(slice_2d)) / (np.ptp(slice_2d) + 1e-8)
        slice_uint8 = slice_norm.astype(np.uint8)


        out_path = SLICE_OUTPUT_DIR / f"{base_name}_slice_sample_{i:02d}.png"
        plt.imsave(out_path, slice_uint8, cmap="gray")

    print(f"  {file.name} : 24 tranches (équidistantes) sauvegardées dans {SLICE_OUTPUT_DIR}")

  ID_0e0fbb4d_ID_e7fa3a566c.nii.gz : 24 tranches (équidistantes) sauvegardées dans /home/tibia/Documents/Brain_GPT_Dataset/slices_png
  ID_00526c11_ID_d6296de728.nii.gz : 24 tranches (équidistantes) sauvegardées dans /home/tibia/Documents/Brain_GPT_Dataset/slices_png
  ID_1423bd41_ID_72db71a879.nii.gz : 24 tranches (équidistantes) sauvegardées dans /home/tibia/Documents/Brain_GPT_Dataset/slices_png


In [ ]:
import nibabel as nib
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Définition des chemins d'entrée et de sortie
INPUT_DIR = Path("/home/tibia/Documents/Brain_GPT_Dataset")
SLICE_OUTPUT_DIR = INPUT_DIR / "slices_png_raw"
SLICE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Liste tous les fichiers NIfTI dans le dossier d'entrée
nii_files = list(INPUT_DIR.glob("*.nii.gz"))

for file in nii_files:
    print(f"Traitement de : {file.name}")
    try:
        # Charge le volume 3D
        img = nib.load(str(file))
        data = img.get_fdata()  # shape (H, W, D) où D est le nombre de tranches
        
        # Le nombre total de tranches dans le volume
        total_slices = data.shape[2]
        
        # Sauvegarde toutes les tranches du volume
        for i in range(total_slices):
            # Sélectionne la tranche 2D sans aucune modification
            slice_2d = data[:, :, i]
            
            # Crée le nom du fichier de sortie
            base_name = file.stem
            out_path = SLICE_OUTPUT_DIR / f"{base_name}_slice_{i:04d}.png"
            
            # Sauvegarde l'image en niveaux de gris sans normalisation.
            # plt.imsave gérera la plage de valeurs en interne.
            plt.imsave(out_path, slice_2d, cmap="gray")

        print(f"  {file.name} : {total_slices} tranches brutes sauvegardées dans {SLICE_OUTPUT_DIR}")

    except Exception as e:
        print(f" ⚠️ Erreur lors du traitement de {file.name}: {e}")

print("\nTraitement terminé.")

In [5]:
SLICE_DIR= Path("/home/tibia/Documents/Brain_GPT_Dataset/slices_png")
print(len(list(SLICE_DIR.glob("*"))))
sorted_files = sorted(SLICE_DIR.glob("*.png"))

# Affichage
for f in sorted_files:
    print(f.name)

72
ID_00526c11_ID_d6296de728.nii_slice_sample_00.png
ID_00526c11_ID_d6296de728.nii_slice_sample_01.png
ID_00526c11_ID_d6296de728.nii_slice_sample_02.png
ID_00526c11_ID_d6296de728.nii_slice_sample_03.png
ID_00526c11_ID_d6296de728.nii_slice_sample_04.png
ID_00526c11_ID_d6296de728.nii_slice_sample_05.png
ID_00526c11_ID_d6296de728.nii_slice_sample_06.png
ID_00526c11_ID_d6296de728.nii_slice_sample_07.png
ID_00526c11_ID_d6296de728.nii_slice_sample_08.png
ID_00526c11_ID_d6296de728.nii_slice_sample_09.png
ID_00526c11_ID_d6296de728.nii_slice_sample_10.png
ID_00526c11_ID_d6296de728.nii_slice_sample_11.png
ID_00526c11_ID_d6296de728.nii_slice_sample_12.png
ID_00526c11_ID_d6296de728.nii_slice_sample_13.png
ID_00526c11_ID_d6296de728.nii_slice_sample_14.png
ID_00526c11_ID_d6296de728.nii_slice_sample_15.png
ID_00526c11_ID_d6296de728.nii_slice_sample_16.png
ID_00526c11_ID_d6296de728.nii_slice_sample_17.png
ID_00526c11_ID_d6296de728.nii_slice_sample_18.png
ID_00526c11_ID_d6296de728.nii_slice_sample_19.p

In [16]:
import os
import shutil

# Définir les chemins des dossiers
source_dir = "/home/tibia/Documents/Brain_GPT_Dataset/slices_png"
target_dir = "/home/tibia/Documents/LLM/BrainGPT/Dataset"

# S'assurer que le dossier cible existe
os.makedirs(target_dir, exist_ok=True)

# Parcourir tous les fichiers dans le dossier source
for filename in os.listdir(source_dir):
    if filename.endswith(".png"):
        # Extraire l'ID de l'étude à partir du nom du fichier
        # Le nom est ID_XXXXXX_ID_YYYYYY...
        parts = filename.split('_')
        if len(parts) >= 2 and parts[0] == "ID":
            study_id = parts[1]
            
            # Créer le chemin du sous-dossier de l'ID
            id_dir = os.path.join(target_dir, study_id)
            os.makedirs(id_dir, exist_ok=True)
            
            # Définir les chemins complets pour le fichier source et cible
            source_path = os.path.join(source_dir, filename)
            target_path = os.path.join(id_dir, filename)
            
            # Déplacer le fichier
            shutil.move(source_path, target_path)
            print(f"Déplacé {filename} vers {id_dir}")

print("Opération terminée.")


Déplacé ID_1423bd41_ID_72db71a879.nii_slice_sample_03.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/1423bd41
Déplacé ID_1423bd41_ID_72db71a879.nii_slice_sample_23.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/1423bd41
Déplacé ID_0e0fbb4d_ID_e7fa3a566c.nii_slice_sample_21.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/0e0fbb4d
Déplacé ID_1423bd41_ID_72db71a879.nii_slice_sample_12.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/1423bd41
Déplacé ID_1423bd41_ID_72db71a879.nii_slice_sample_22.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/1423bd41
Déplacé ID_0e0fbb4d_ID_e7fa3a566c.nii_slice_sample_22.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/0e0fbb4d
Déplacé ID_00526c11_ID_d6296de728.nii_slice_sample_22.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/00526c11
Déplacé ID_0e0fbb4d_ID_e7fa3a566c.nii_slice_sample_07.png vers /home/tibia/Documents/LLM/BrainGPT/Dataset/0e0fbb4d
Déplacé ID_1423bd41_ID_72db71a879.nii_slice_sample_15.png vers /home/tibia/Docum

In [9]:
import os
import json
from tqdm import tqdm
from glob import glob

dataset_path = "/home/tibia/Documents/LLM/BrainGPT/Dataset/merged"

In [17]:
import os
import shutil
from glob import glob

dataset_path = "/home/tibia/Documents/LLM/BrainGPT/Dataset"
merged_path = os.path.join(dataset_path, "merged")

os.makedirs(merged_path, exist_ok=True)

# Cherche tous les PNG dans tous les sous-dossiers
png_files = glob(os.path.join(dataset_path, "*", "*.png"))

for file_path in png_files:
    # Copie le fichier dans merged en gardant le même nom
    shutil.copy(file_path, merged_path)

print(f"Copied {len(png_files)} files to {merged_path}")

Copied 72 files to /home/tibia/Documents/LLM/BrainGPT/Dataset/merged


In [18]:
import os
from pathlib import Path

dataset_path = Path("/home/tibia/Documents/LLM/BrainGPT/Dataset/merged")

# Liste tous les fichiers png dans le dossier merged
png_files = sorted(dataset_path.glob("*.png"))

for f in png_files:
    old_name = f.name  # ex: ID_441cea72_ID_4b13324a92.nii_slice00.png vient du RSNA

    parts = old_name.split("_")
    if len(parts) < 2:
        continue

    # On garde seulement les chiffres pour la première partie après ID pour que loadimage marche
    first_id = ''.join(filter(str.isdigit, parts[1]))  # '441cea72' -> '44172'

    # On fait meme pour la deuxieme partie
    second_id = ''.join(filter(str.isdigit, parts[3].split(".")[0]))  # '4b13324a92' -> '4132492'

    # Slice et extension
    slice_and_ext = "_".join(parts[4:])  # ex: slice00.png

    # Nouveau nom
    new_name = f"ID_{first_id}_ID_{second_id}_{slice_and_ext}"
    new_path = f.parent / new_name

    print(f"Renaming: {old_name} -> {new_name}")
    f.rename(new_path)

Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_00.png -> ID_0052611_ID_6296728_slice_sample_00.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_01.png -> ID_0052611_ID_6296728_slice_sample_01.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_02.png -> ID_0052611_ID_6296728_slice_sample_02.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_03.png -> ID_0052611_ID_6296728_slice_sample_03.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_04.png -> ID_0052611_ID_6296728_slice_sample_04.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_05.png -> ID_0052611_ID_6296728_slice_sample_05.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_06.png -> ID_0052611_ID_6296728_slice_sample_06.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_07.png -> ID_0052611_ID_6296728_slice_sample_07.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_08.png -> ID_0052611_ID_6296728_slice_sample_08.png
Renaming: ID_00526c11_ID_d6296de728.nii_slice_sample_09

In [1]:
# Code pour faire le .json d'instruction 

import os
import json
import glob
from collections import defaultdict


# Chemin vers le dossier contenant les images brutes
dataset_path = "/home/tibia/Documents/LLM/BrainGPT/Dataset/black"

# Nom court du dataset (doit être le même que dans main.py)
short_name = "Adri"

# L'instruction à donner au modèle
INSTRUCTION_TEXT = (
    "You are an AI assistant specialized in radiology topics. "
    "You are provided with brain CT slices from a single study. "
    "The number of slices is {num_slices}. "
    "Please generate medical descriptions based on the images in a consistent style. "
    "Use the following guidelines: - Degree: Indicate the intensity or state (e.g., normal, mild, chronic, old, etc). "
    "- Landmark: Specify the area of interest (e.g., intracerebral, midline, parenchyma, sulci, etc). "
    "- Feature: Describe any observed abnormalities (e.g., hemorrhage, atrophy, infarcts, etc). "
    "- Impression: Conclude with a clinical impression (e.g., arteriosclerotic encephalopathy, intracerebral hemorrhage, dementia, etc). "
    "Ensure consistency and clarity in the report."
)

# =================================================

def generate_instruction_json():
    # 1. Lister les images
    extensions = ["*.png", "*.jpg", "*.jpeg", "*.bmp"]
    all_files = []
    for ext in extensions:
        all_files.extend(glob.glob(os.path.join(dataset_path, ext)))
    
    if not all_files:
        all_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) 
                     if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

    # 2. Grouper les images par "Étude" (Patient ID)
    studies = defaultdict(list)

    print(f"Traitement de {len(all_files)} fichiers...")

    for file_path in all_files:
        filename = os.path.basename(file_path)
        
        # --- Etape A : Générer l'ID unique de l'image (pour le lien vers le Registry) ---
        name_without_ext = os.path.splitext(filename)[0]
        clean_name = name_without_ext.replace(".nii", "").replace(".", "_")
        full_image_id = f"{short_name}_IMG_{clean_name}"
        
        # --- Etape B : Identifier l'étude (Patient) ---
        # Ex: ID_1423bd41_ID_72db71a879.nii_slice_sample_01.png
        # On coupe avant "_slice_sample_" pour avoir le nom du patient
        if "_slice_sample_" in filename:
            study_id_raw = filename.split("_slice_sample_")[0]
            
            # Récupération du numéro de slice pour le tri
            try:
                slice_num_str = filename.split("_slice_sample_")[-1].split(".")[0]
                slice_num = int(slice_num_str)
            except:
                slice_num = 0
            
            studies[study_id_raw].append((slice_num, full_image_id))
        else:
            print(f"Ignoré (format incorrect) : {filename}")

    # 3. Construire le JSON final
    json_output = {
        "meta": {
            "version": "0.0.2",
            "time": "2024-02",
            "author": "big_data_center"
        },
        "data": {}
    }

    print(f"Génération des instructions pour {len(studies)} études...")

    for study_id_raw, slices in studies.items():
        # IMPORTANT : Trier les slices dans l'ordre (0, 1, 2...)
        slices.sort(key=lambda x: x[0])
        sorted_image_ids = [item[1] for item in slices]
        
        # MODIFICATION ICI : Création de la clé basée sur le nom réel
        # On nettoie un peu le nom (enlève .nii et remplace les points par _)
        # Ex: ID_1423bd41.nii -> ID_1423bd41
        clean_study_key = study_id_raw.replace(".nii", "").replace(".", "_")
        
        # On peut ajouter un préfixe pour faire propre, ou laisser tel quel.
        # Ici je mets "INS_" + le nom réel pour indiquer que c'est une instruction.
        ins_key = f"INS_{clean_study_key}" 
        
        # Mise à jour du texte avec le vrai nombre de slices
        current_instruction = INSTRUCTION_TEXT.format(num_slices=len(sorted_image_ids))

        json_output["data"][ins_key] = {
            "instruction": current_instruction,
            "answer": "",
            "image_ids": sorted_image_ids,
            "rel_ins_ids": []
        }

    # 4. Sauvegarde
    output_filename = "instruction_dataset.json"
    with open(output_filename, "w") as f:
        json.dump(json_output, f, indent=4)

    print(f"Terminé ! Fichier '{output_filename}' généré.")

if __name__ == "__main__":
    generate_instruction_json()

Traitement de 24 fichiers...
Génération des instructions pour 1 études...
Terminé ! Fichier 'instruction_dataset.json' généré.


In [4]:
import json

# Chemins de tes fichiers (basés sur tes logs)
img_json_path = "/home/tibia/Documents/LLM/BrainGPT/data/Adri_2.json"
instruct_json_path = "/home/tibia/Documents/LLM/BrainGPT/data/instruction_dataset.json"

print("--- Chargement des fichiers ---")
with open(img_json_path, 'r') as f:
    img_data = json.load(f)

with open(instruct_json_path, 'r') as f:
    inst_data = json.load(f)

print(f"\nNombre d'images dans Adri_2.json : {len(img_data)}")
# On prend la première clé disponible pour voir sa tête
first_img_key = list(img_data.keys())[0] if img_data else "VIDE"
print(f"Exemple de clé dans Adri_2.json      : '{first_img_key}'")

print(f"\nNombre d'instructions : {len(inst_data['data'])}")
# On prend le premier ID image demandé dans la première instruction
first_inst_key = list(inst_data['data'].keys())[0]
first_requested_id = inst_data['data'][first_inst_key]['image_ids'][0]
print(f"Exemple d'ID demandé dans instructions : '{first_requested_id}'")

print("\n--- Verdict ---")
if first_img_key == first_requested_id:
    print(" Les clés correspondent ! (Le problème est ailleurs ou sur une image spécifique)")
else:
    print(" LES CLÉS NE CORRESPONDENT PAS.")
    print("Il faut regénérer Adri_2.json avec la bonne logique de nommage.")

--- Chargement des fichiers ---

Nombre d'images dans Adri_2.json : 24
Exemple de clé dans Adri_2.json      : 'Adri_IMG_ID_1423bd41_ID_72db71a879_slice_sample_03'

Nombre d'instructions : 1
Exemple d'ID demandé dans instructions : 'Adri_IMG_ID_1423bd41_ID_72db71a879_slice_sample_01'

--- Verdict ---
 LES CLÉS NE CORRESPONDENT PAS.
Il faut regénérer Adri_2.json avec la bonne logique de nommage.
